In [1]:
'''
CREATE OR REPLACE FUNCTION public.create_user(
    email text,
    password text
) RETURNS void AS $$
  declare
  user_id uuid;
  encrypted_pw text;
BEGIN
  user_id := gen_random_uuid();
  encrypted_pw := crypt(password, gen_salt('bf'));
  
  INSERT INTO auth.users
    (instance_id, id, aud, role, email, encrypted_password, email_confirmed_at, recovery_sent_at, last_sign_in_at, raw_app_meta_data, raw_user_meta_data, created_at, updated_at, confirmation_token, email_change, email_change_token_new, recovery_token)
  VALUES
    ('00000000-0000-0000-0000-000000000000', user_id, 'authenticated', 'authenticated', email, encrypted_pw, '2023-05-03 19:41:43.585805+00', '2023-04-22 13:10:03.275387+00', '2023-04-22 13:10:31.458239+00', '{"provider":"email","providers":["email"]}', '{}', '2023-05-03 19:41:43.580424+00', '2023-05-03 19:41:43.585948+00', '', '', '', '');
  
  INSERT INTO auth.identities (id, user_id, identity_data, provider, last_sign_in_at, created_at, updated_at)
  VALUES
    (gen_random_uuid(), user_id, format('{"sub":"%s","email":"%s"}', user_id::text, email)::jsonb, 'email', '2023-05-03 19:41:43.582456+00', '2023-05-03 19:41:43.582497+00', '2023-05-03 19:41:43.582497+00');
END;
$$ LANGUAGE plpgsql;
'''

'\nCREATE OR REPLACE FUNCTION public.create_user(\n    email text,\n    password text\n) RETURNS void AS $$\n  declare\n  user_id uuid;\n  encrypted_pw text;\nBEGIN\n  user_id := gen_random_uuid();\n  encrypted_pw := crypt(password, gen_salt(\'bf\'));\n  \n  INSERT INTO auth.users\n    (instance_id, id, aud, role, email, encrypted_password, email_confirmed_at, recovery_sent_at, last_sign_in_at, raw_app_meta_data, raw_user_meta_data, created_at, updated_at, confirmation_token, email_change, email_change_token_new, recovery_token)\n  VALUES\n    (\'00000000-0000-0000-0000-000000000000\', user_id, \'authenticated\', \'authenticated\', email, encrypted_pw, \'2023-05-03 19:41:43.585805+00\', \'2023-04-22 13:10:03.275387+00\', \'2023-04-22 13:10:31.458239+00\', \'{"provider":"email","providers":["email"]}\', \'{}\', \'2023-05-03 19:41:43.580424+00\', \'2023-05-03 19:41:43.585948+00\', \'\', \'\', \'\', \'\');\n  \n  INSERT INTO auth.identities (id, user_id, identity_data, provider, last_sign

In [4]:
from uuid import uuid4 # built-in / std lib include
from faker import Faker
import bcrypt
from IPython.display import display
import json

fake = Faker()

print("DELETE FROM auth.users;")
print("DELETE FROM auth.identities;")
print()

class User:
    def __init__(self, i):
        self.id_ = str(uuid4())
        #self.email = fake.email()
        self.email = f"{i}@nyu.edu"
        #self.password = fake.password()
        self.password = "password"
        self.hashed_password = self.hash_password(self.password)
        print(
            '/*', 
            json.dumps({"email": self.email, "password": self.password}),
            '*/',
        )
        # uuid
        print(f"/* {self.id_} */")

    @staticmethod
    def hash_password(pwd: str):
        bytes_ = pwd.encode('utf-8')
        salt = bcrypt.gensalt(rounds=10, prefix=b"2a")
        hash = bcrypt.hashpw(bytes_, salt)
        return hash.decode()

    def auth_user(self) -> str:
        s = f"""('00000000-0000-0000-0000-000000000000', '{self.id_}', 'authenticated', 'authenticated', '{self.email}', '{self.hashed_password}', '2023-05-03 19:41:43.585805+00', NULL, '', NULL, '', '2023-04-22 13:10:03.275387+00', '', '', NULL, '2023-04-22 13:10:31.458239+00', '{{"provider":"email","providers":["email"]}}', '{{}}', NULL, '2023-05-03 19:41:43.580424+00', '2023-05-03 19:41:43.585948+00', NULL, NULL, '', '', NULL, '', 0, NULL, '', NULL)"""
        return s

    def auth_identity(self) -> str:
        s = f"""('{self.id_}', '{self.id_}'::uuid, '{{"sub":"{self.id_}","email":"{self.email}"}}', 'email', '2023-05-03 19:41:43.582456+00', '2023-05-03 19:41:43.582497+00', '2023-05-03 19:41:43.582497+00')"""
        return s

    def __repr__(self):
        return self.single_row()


if __name__ == "__main__":

    users: list[User] = [User(i) for i in range(1, 16)]

    print("""INSERT INTO auth.users (instance_id, id, aud, role, email, encrypted_password, email_confirmed_at, invited_at, confirmation_token, confirmation_sent_at, recovery_token, recovery_sent_at, email_change_token_new, email_change, email_change_sent_at, last_sign_in_at, raw_app_meta_data, raw_user_meta_data, is_super_admin, created_at, updated_at, phone, phone_confirmed_at, phone_change, phone_change_token, phone_change_sent_at, email_change_token_current, email_change_confirm_status, banned_until, reauthentication_token, reauthentication_sent_at)\n  VALUES""")
    for index, user in enumerate(users):
        line_terminator = ("," if index is not (len(users)-1) else ";")
        print(f"    {user.auth_user()}{line_terminator}")

    print("""INSERT INTO auth.identities (provider_id,user_id,identity_data,provider,last_sign_in_at,created_at,updated_at)\n  VALUES""")
    for index, user in enumerate(users):
        line_terminator = ("," if index is not (len(users) - 1) else ";")
        print(f"    {user.auth_identity()}{line_terminator}")
print()

DELETE FROM auth.users;
DELETE FROM auth.identities;

/* {"email": "1@nyu.edu", "password": "password"} */
/* c9f0d135-aa05-45c9-8cf2-5570eb062b58 */
/* {"email": "2@nyu.edu", "password": "password"} */
/* 3e8d9652-c750-4464-9885-697ae0946290 */
/* {"email": "3@nyu.edu", "password": "password"} */
/* cafa7e71-0c42-4037-838f-19011983283e */
/* {"email": "4@nyu.edu", "password": "password"} */
/* 6a69d722-d493-4e28-8fd0-b6f8ceef979b */
/* {"email": "5@nyu.edu", "password": "password"} */
/* 858a24a9-51f7-4275-95e0-b507494c8dc0 */
/* {"email": "6@nyu.edu", "password": "password"} */
/* 274a1a84-0b2c-4181-a932-6ca2c9e1863d */
/* {"email": "7@nyu.edu", "password": "password"} */
/* ceacb77b-07c6-46d0-9080-f65f70a1eadc */
/* {"email": "8@nyu.edu", "password": "password"} */
/* bda47b2b-d2b3-4d13-88d6-dd1883a6ebe7 */
/* {"email": "9@nyu.edu", "password": "password"} */
/* 112263f7-dfd3-4912-80f9-963ec098b42f */
/* {"email": "10@nyu.edu", "password": "password"} */
/* 0b3eb263-6488-4528-8f5b-8

In [3]:
# 虽然密码和salt都一样，这个函数每次生成的哈希还是会不同
# 因为使用了rounds=10

# 哈希值中存储了第几轮的信息，checkpw解密时会用到
print(bcrypt.gensalt(rounds=10, prefix=b"2a"))
print(bcrypt.gensalt(rounds=10, prefix=b"2a"))


for _ in range(3):
    pwd = "password"

    hashed_pwd = bcrypt.hashpw(
        pwd.encode(),
        bcrypt.gensalt(rounds=10, prefix=b"2a")
    ).decode()

    verified = bcrypt.checkpw(pwd.encode(), hashed_pwd.encode())
    print(verified, pwd, hashed_pwd)



b'$2a$10$qlUKgA/E8Y5O.ibpBFaVBO'
b'$2a$10$3Vyfq/MkriE6W.PH7HVqpO'
True password $2a$10$MhDQK7LeaDRiCt6Iod/.fehBzB0c93n00Uj4CUq4CwwT05aCwhxJq
True password $2a$10$XhFq0GxgijZ00yOLvRNWOO.Q25y5CEgTVXT3uw0ToxGAZQ.4GvL9.
True password $2a$10$ga2LX4qHzQdXgIG/P8BS4eDxCjr4cCqCr7VYXGiOyyQCPx4w7mKDS
